In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import json
import os
import datetime
import nrrd
import bg_space as bg
from pprint import pprint
from matplotlib.colors import LogNorm, Normalize
import seaborn as sb
import pathlib
from bg_atlasapi import BrainGlobeAtlas
from lcm_registration import lcm_registration_functions as lrf
from lcm_registration import visualign_functions as vis
from znamutils import slurm_it
from PIL import Image
import os
import yaml
%matplotlib inline

(Optional) Before using QuickNII, you can run the below code to make LCM section images conform to requirements for lcm registration pipeline (convert images to jpg and making sure they start with '_s0')

In [ ]:
input_folder = "/nemo/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6d/LCM/sections_same_orientation/"
output_folder = "/nemo/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/FIAA45.6d/LCM/sections_QuickNII/"

lrf.convert_tif_to_jpg(input_folder, output_folder)

In [ ]:
# remove mouse name from section images filename.
mouse_name = "FIAA45.6D"
for filename in os.listdir(input_folder):
    if filename.startswith(f"{mouse_name}_") and filename.endswith(".TIF"):
        new_filename = filename.replace(f"{mouse_name}_", "")
        os.rename(
            os.path.join(input_folder, filename),
            os.path.join(input_folder, new_filename),
        )
        print(f"Renamed {filename} to {new_filename}")

After completing registration using QuickNII and Visualign softwhare, you can now calculate z distance between sections

In [ ]:
mouse = 'FIAA55.4d'
parameters_path = (
    f"/camp/lab/znamenskiyp/home/shared/projects/turnerb_A1_MAPseq/{mouse}/Sequencing"
)

1. Convert each registered section image to numpy file containing 2D allen ccf coordinates

In [ ]:
convert_job = lrf.convert_images(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

2. Now find find the pixels in the previous section with closest Euclidean distance, in order to calculate z-distance between the two sections

In [ ]:
lrf.get_euclidean_distance(
    parameters_path=parameters_path,
    use_slurm=True,
    job_dependency=convert_job,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

(Optional) check how the z distance between sections looks

In [ ]:

parameters =lrf.load_parameters(directory=parameters_path)
section = 44
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
lrf.check_z_dist(lcm_dir=pathlib.Path(parameters["lcm_directory"]), section=section, fig=fig, axs=axs)


3. Now send job to create 3D atlas containing all LCM ROIs

In [ ]:
lrf.get_roi_vol(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)

4. Calculate volumes and generate brain region annotations for each sample

In [ ]:
lrf.group_ROI_coordinates(
    parameters_path=parameters_path,
    use_slurm=True,
    slurm_folder="/camp/home/turnerb/slurm_logs",
)